In [1]:
from pathlib import Path
import re
import warnings

import numpy as np
import pandas as pd
import matplotlib_venn
import matplotlib.pyplot as plt

import bioquest as bq
import genekit as gk

In [2]:
warnings.filterwarnings(action="ignore")
OUTPUT_DIR = "data"
Path(OUTPUT_DIR).mkdir(parents=True,exist_ok=True)

## TCGA数据

In [3]:
df = bq.tl.get_TCGA_mRNA("/Volumes/T7/DataHub/TCGA/mRNA/TCGA_COAD_mRNA.arrow", dtype='tpm', label="[01]1A", gene_type=None, barcode_length=16)

In [4]:
df.loc[["ENSG00000197837"],"gene_name"] = ["H4C16"]

In [5]:
df.set_index(keys="gene_name", drop=True, inplace=True)
df.drop(columns="gene_type", inplace=True)

In [6]:
df = bq.tl.unique_exprs(df)

In [7]:
df = bq.tl.unique_exprs(df.T,np.mean).T

In [9]:
df.reset_index().to_feather(f"{OUTPUT_DIR}/TCGA_COAD_mRNA.arrow")

In [8]:
df.to_csv(f"{OUTPUT_DIR}/TCGA_COAD_mRNA.csv.gz")

## cluster heatmap

In [ ]:
tpm=pd.read_csv("results/5.model/clinical/exprs.csv.gz",index_col=0)

In [ ]:
group=pd.read_csv('HM019/结果/四 基于WGCNA识别昼夜节律相关模块和关键基因/分数最佳分组.csv',index_col=0)
group.sort_values(by='Score',inplace=True)
samples = group.index.to_list()

In [ ]:
tpm.columns = bq.st.subs(string=tpm.columns,start=0,stop=12)

In [ ]:
df=pd.read_csv("HM019/结果/二 识别差异表达的昼夜节律相关基因/40个交集基因.csv",index_col=0)
tpm = tpm.loc[df.index,samples]

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np
exprs=StandardScaler().fit_transform(tpm.T)
exprs=pd.DataFrame(exprs.T,columns=tpm.columns,index=tpm.index)

In [ ]:
import numpy as np
import pandas as pd

import PyComplexHeatmap
from PyComplexHeatmap import *
import matplotlib.pylab as plt
plt.rcParams['figure.dpi'] = 120
plt.rcParams['savefig.dpi']=300

In [ ]:
top_ha = HeatmapAnnotation(
	Score=anno_simple(group.Score,add_text=True, colors={'low': bq.pl.Palette.set2[0], 'high': bq.pl.Palette.set2[1]}),
    Sex=anno_simple(group.Sex, colors={'MALE': bq.pl.Palette.set2[2], 'FEMALE': bq.pl.Palette.set2[3]}),
    Age=anno_simple(group.Age, colors={'Age<60': bq.pl.Palette.set2[4], 'Age>=60': bq.pl.Palette.set2[5]}),
    Grade = anno_simple(group.Grade, colors={x: bq.pl.Palette.economist[y] for x,y in zip(["G1","G2","G3","G4"],(0,1,2,3))},legend="G***")
	)

In [ ]:
plt.figure(figsize=(6, 6))                         
cm = ClusterMapPlotter(
    data=exprs,
    # vmax=3,
    # standard_scale=True,
    # z_score = True,
    show_rownames = True,
    show_colnames = False,
    top_annotation=top_ha, 
    col_split=2, 
    col_cluster=False,
    col_dendrogram=False,
    row_dendrogram = True,
    row_split=None, 
    col_split_gap=0.5,
    row_split_gap=1,
    cmap='viridis',
    plot=True,
    tree_kws={'col_cmap': 'turbo', 'row_cmap': 'turbo'}
    );
plt.savefig("HM019/结果/三 识别昼夜节律相关评分（CR-score）组/cluster_heatmap.pdf", bbox_inches='tight')
plt.show()

In [ ]:
import scipy.stats as spss

In [ ]:
tb = pd.crosstab(group.Score, group.Sex)
spss.chi2_contingency(tb)

In [ ]:
tb = pd.crosstab(group.Score, group.Age)
spss.chi2_contingency(tb)

In [ ]:
tb = pd.crosstab(group.Score, group.Grade)
spss.chi2_contingency(tb)

## GSEA

In [ ]:
df=pd.read_csv("results/5.model/clinical/exprs.csv.gz",index_col=0)
gr=pd.read_csv("results/5.model/clinical/cluster.csv",index_col=0)
df=df.loc[:,gr.index]

In [ ]:
def gsea(frame,group,gene_sets,n_jobs=8):
    import gseapy
    res = gseapy.gsea(data=frame, # row -> genes, column-> samples
        gene_sets=gene_sets,
        cls=group,
        permutation_num=1000,
        permutation_type='phenotype',
        outdir=None,
        seed=1314,
        method='signal_to_noise', # signal_to_noise
        threads= n_jobs)
    
    return res

In [ ]:
df.shape

In [ ]:
clss = pd.Categorical(gr.Group,categories=["High","Low"],ordered=True)

In [ ]:
gs=("GO_Biological_Process_2021","GO_Molecular_Function_2021","GO_Cellular_Component_2021","KEGG_2021_Human","MSigDB_Hallmark_2020")
res = [gsea(df,clss,gene_sets=x).res2d for x in gs]

In [ ]:
df = pd.concat(res,keys=["GOBP","GOMF","GOCC","KEGG","Hallmark"])
df.drop(columns="Name", inplace=True)
df.Term=bq.st.removes(string=df.Term,pattern=r" \(.+\)")

In [ ]:
df.to_csv("results/5.model/GSEA/gsea.csv")

In [ ]:
df.index = df.index.droplevel(1)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
import gseapy
for x,y in zip(("GO Biological Process","GO Molecular Function","GO Cellular Component","KEGG","Hallmark"),("GOBP","GOMF","GOCC","KEGG","Hallmark")):
    ax = gseapy.dotplot(df.loc[y:],
             column="FDR q-val",
             title=x,
             size=3, # adjust dot size
             top_term=15,
             figsize=(4,5), cutoff=0.25, show_ring=False)
    plt.savefig(f"results/5.model/GSEA/GSEA_{y}.pdf",bbox_inches='tight', dpi=300);